In [ ]:
import mne
import datetime
import json
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy import stats

sub_ses_dict={}
subjs=list(range(1,28))
for i in subjs:
    try:
        subj='{:02d}'.format(i)
        raw_path1=f'/hpc2hdd/home/yyang937/datasets/gwilliams2023/download/sub-{subj}/ses-0/meg/sub-{subj}_ses-0_task-1_meg.con'
        raw_path2=f'/hpc2hdd/home/yyang937/datasets/gwilliams2023/download/sub-{subj}/ses-1/meg/sub-{subj}_ses-1_task-1_meg.con'

        meg = mne.io.read_raw_kit(raw_path1, preload=False, verbose=False)
        a=meg.info['meas_date']
        meg = mne.io.read_raw_kit(raw_path2, preload=False, verbose=False)
        b=meg.info['meas_date']
        a = datetime.datetime(a.year, a.month, a.day)
        b = datetime.datetime(b.year, b.month, b.day)
        delta = b - a
        days = delta.days
        sub_ses_dict[i]=days
        del a,b,days,meg,delta
    except Exception as e:
        pass
meta_output_path='/hpc2hdd/home/yyang937/datasets/gwilliams2023/preprocess5/split2/interval_days.json'
with open(meta_output_path, "w") as json_file:
    json.dump(sub_ses_dict, json_file, indent=4,ensure_ascii=False)

In [ ]:
path='/hpc2hdd/home/yyang937/brain2text/whisper/output_models/10-gwilliams2023-split2/whisper-base/checkpoint-final/save_subjects_evaluation_data.json'
with open(path, 'r', encoding='utf-8') as f:
    save_subjects_evaluation_data = json.load(f)
subj_results_dict=save_subjects_evaluation_data['subj_results_dict']
days=save_subjects_evaluation_data['days']
subjects=save_subjects_evaluation_data['subjects']
eval_keys=save_subjects_evaluation_data['eval_keys']
metrics_pearson_r={}
metrics_spearman_r={}
plt.figure(figsize=(6,4))
for eval_key in eval_keys:
    if eval_key.startswith('bleu'):
        for subj in subjects:
            subj_results_dict[subj][eval_key]=subj_results_dict[subj][eval_key]*100
    subj_results_key_list=[subj_results_dict[subj][eval_key] for subj in subjects]
    pearson_r, _ = stats.pearsonr(days, subj_results_key_list)
    metrics_pearson_r[eval_key]=pearson_r
    spearmanr, _ = stats.spearmanr(days, subj_results_key_list)
    metrics_spearman_r[eval_key]=spearmanr
    plt.scatter(days,[subj_results_dict[subj][eval_key] for subj in subjects],s=15)
print(metrics_pearson_r)
print(metrics_spearman_r)
plt.legend(eval_keys)
plt.xlabel('interval days')
plt.ylabel('performance')
plt.savefig('/hpc2hdd/home/yyang937/brain2text/whisper/output_models/10-gwilliams2023-split2/whisper-base/checkpoint-final/subjects_metrics.pdf')
plt.show()
plt.close()

In [ ]:
import statistics
from scipy import stats
import jsonlines
import pandas as pd
def read_jsonlines(file_path):
    json_dicts = []
    with jsonlines.open(file_path, mode='r') as reader:
        for json_obj in reader:
            json_dicts.append(json_obj)
    return json_dicts
data=read_jsonlines('/hpc2hdd/home/yyang937/brain2text/whisper/output_models/10-gwilliams2023-split2/whisper-base/checkpoint-final/test_each_results_and_metrics.jsonl')
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
# print(pd.DataFrame(data[:4]))
def anova(data):
    for i,d in enumerate(data):
        d['bleu_1']=d['metrics']['bleu-1']*100
        d['sent_len']=len(d['sentence'].split())
        # d['day_type']=
    df = pd.DataFrame(data)
    model = ols('bleu_1 ~ C(subj)+C(story)+C(voice)+C(sent_len) + C(speech_rate)+C(voice):C(story)+C(sent_len):C(story)+C(sent_len):C(voice)+C(sent_len):C(speech_rate)+ C(voice):C(speech_rate)+ C(story):C(speech_rate) + C(subj):C(speech_rate)', data=df).fit() 
    results = anova_lm(model)
    print(results)
anova(data)

In [ ]:
import statistics
from scipy import stats
import jsonlines
import pandas as pd
def read_jsonlines(file_path):
    json_dicts = []
    with jsonlines.open(file_path, mode='r') as reader:
        for json_obj in reader:
            json_dicts.append(json_obj)
    return json_dicts
data=read_jsonlines('/hpc2hdd/home/yyang937/brain2text/whisper/output_models/10-gwilliams2023-split2/whisper-base/checkpoint-final/test_each_results_and_metrics.jsonl')
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
meta_output_path='/hpc2hdd/home/yyang937/datasets/gwilliams2023/preprocess5/split2/interval_days.json'
with open(meta_output_path, "r") as json_file:
    sub_ses_dict=json.load(json_file)
# 研究间隔时间和
# print(pd.DataFrame(data[:4]))
def piecewise_function(n):
    if n<7:
        return 1
    elif n<30:
        return 2
    elif n<150:
        return 3
    elif n<160:
        return 4
    elif n<170:
        return 5
    else:
        return 6
def anova(data,sub_ses_dict=sub_ses_dict):
    new_data=[]
    sub_ses_dict={int(key):value for key,value in sub_ses_dict.items()}
    for i,d in enumerate(data):
        # print(d['subj'],sub_ses_dict.keys())
        if d['subj'] not in sub_ses_dict.keys():
            continue
        
        d['bleu_1']=d['metrics']['bleu-1']
        d['day_intervals']=piecewise_function(sub_ses_dict[d['subj']])
        new_data.append(d)
    df = pd.DataFrame(new_data)
    # print(df)
    model = ols('bleu_1 ~ C(subj) + C(day_intervals) + C(subj):C(day_intervals)', data=df).fit() 
    results = anova_lm(model)
    print(results)
anova(data)

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns
data=read_jsonlines('/hpc2hdd/home/yyang937/brain2text/whisper/output_models/10-gwilliams2023-split2/whisper-base/checkpoint-final/test_each_results_and_metrics.jsonl')

def regression_plot_by_subject(data):
    for i,d in enumerate(data):
        d['bleu_1']=d['metrics']['bleu-1']*100

    df = pd.DataFrame(data)

    fig, ax = plt.subplots(figsize=(10,8),dpi=600)
    
    for subj in df['subj'].unique():

        subj_data = df[df['subj']==subj]

        X = subj_data['speech_rate'].values.reshape(-1,1)
        y = subj_data['bleu_1'].values#*100

        regr = LinearRegression().fit(X, y)

        ax.plot(X, regr.predict(X), color='C'+str(int(subj)), 
                linestyle='-', linewidth=3)

        ax.scatter(X, y, alpha=0.2, s=10,
                  color='C'+str(int(subj)), label='Subject '+str(subj))
    X = df['speech_rate'].values.reshape(-1,1)
    y = df['bleu_1'].values#*100

    regr = LinearRegression().fit(X, y)
    print(regr.coef_)
    ax.plot(X, regr.predict(X), color='green', 
            linestyle='-', linewidth=100,alpha=0.2)
    # ax.legend(loc='best')
    ax.set_xlabel('Speech Rate')
    ax.set_ylabel('BLEU Score')
    ax.set_ylim([0.125*100,0.25*100])
    ax.set_xlim([144.5,205.5])

    sns.despine()
    # plt.legend(eval_keys)
    plt.xlabel('speech rate')
    plt.ylabel('BLEU-1')
    plt.savefig('/hpc2hdd/home/yyang937/brain2text/whisper/output_models/10-gwilliams2023-split2/whisper-base/checkpoint-final/speechrate_metrics.pdf')

    plt.show()
regression_plot_by_subject(data)